In [ ]:
import pandas as pd
import requests
import io
import os
import numpy as np

In [ ]:
store = [["burgers","meatballs","eggs"],
      ["turkey","burgers","mineral water","cooking oil"],
      ["burgers","grated cheese","pasta","honey","toothpaste"],
      ["meatballs","milk","eggs","honey","french fries","protein bar"],
      ["red wine","shrimp","pasta","pepper","eggs","chocolate","shampoo"],
      ["burgers","black tea","green tea"],
      ["burgers","black tea","green tea"],
      ["burgers","meatballs","eggs"],
      ["burgers","meatballs","eggs"],
      ["burgers","meatballs","eggs"],
      ["escalope","burgers","shallot"],
      ["red wine","shrimp","pasta","pepper","eggs","chocolate","shampoo"],
      ["spaghetti","milk","french fries","green tea"]]

In [ ]:
store

[['burgers', 'meatballs', 'eggs'],
 ['turkey', 'burgers', 'mineral water', 'cooking oil'],
 ['burgers', 'grated cheese', 'pasta', 'honey', 'toothpaste'],
 ['meatballs', 'milk', 'eggs', 'honey', 'french fries', 'protein bar'],
 ['red wine', 'shrimp', 'pasta', 'pepper', 'eggs', 'chocolate', 'shampoo'],
 ['burgers', 'black tea', 'green tea'],
 ['burgers', 'black tea', 'green tea'],
 ['burgers', 'meatballs', 'eggs'],
 ['burgers', 'meatballs', 'eggs'],
 ['burgers', 'meatballs', 'eggs'],
 ['escalope', 'burgers', 'shallot'],
 ['red wine', 'shrimp', 'pasta', 'pepper', 'eggs', 'chocolate', 'shampoo'],
 ['spaghetti', 'milk', 'french fries', 'green tea']]

## 先驗演算法(Apriori Algorithm)



In [ ]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

In [ ]:
apriori_Encoder = TransactionEncoder()
apriori_Encoder_Array = apriori_Encoder.fit(store).transform(store)

In [ ]:
apriori_Encoder_Array

array([[False,  True, False, False,  True, False, False, False, False,
        False,  True, False, False, False, False, False, False, False,
        False, False, False, False, False],
       [False,  True, False,  True, False, False, False, False, False,
        False, False, False,  True, False, False, False, False, False,
        False, False, False, False,  True],
       [False,  True, False, False, False, False, False,  True, False,
         True, False, False, False,  True, False, False, False, False,
        False, False, False,  True, False],
       [False, False, False, False,  True, False,  True, False, False,
         True,  True,  True, False, False, False,  True, False, False,
        False, False, False, False, False],
       [False, False,  True, False,  True, False, False, False, False,
        False, False, False, False,  True,  True, False,  True, False,
         True,  True, False, False, False],
       [ True,  True, False, False, False, False, False, False,  True,

In [ ]:
apriori_Encoder.columns_

['black tea',
 'burgers',
 'chocolate',
 'cooking oil',
 'eggs',
 'escalope',
 'french fries',
 'grated cheese',
 'green tea',
 'honey',
 'meatballs',
 'milk',
 'mineral water',
 'pasta',
 'pepper',
 'protein bar',
 'red wine',
 'shallot',
 'shampoo',
 'shrimp',
 'spaghetti',
 'toothpaste',
 'turkey']

In [ ]:
df = pd.DataFrame(apriori_Encoder_Array, columns=apriori_Encoder.columns_)
df

,black tea,burgers,chocolate,cooking oil,eggs,escalope,french fries,grated cheese,green tea,honey,meatballs,milk,mineral water,pasta,pepper,protein bar,red wine,shallot,shampoo,shrimp,spaghetti,toothpaste,turkey
0,False,True,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
1,False,True,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True
2,False,True,False,False,False,False,False,True,False,True,False,False,False,True,False,False,False,False,False,False,False,True,False
3,False,False,False,False,True,False,True,False,False,True,True,True,False,False,False,True,False,False,False,False,False,False,False
4,False,False,True,False,True,False,False,False,False,False,False,False,False,True,True,False,True,False,True,True,False,False,False
5,True,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
6,True,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
7,False,True,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
8,False,True,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
9,False,True,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False


In [ ]:
apriori(df, min_support=0.2, use_colnames=True)

,support,itemsets
0,0.692308,(burgers)
1,0.538462,(eggs)
2,0.230769,(green tea)
3,0.384615,(meatballs)
4,0.230769,(pasta)
5,0.307692,"(burgers, eggs)"
6,0.307692,"(burgers, meatballs)"
7,0.384615,"(meatballs, eggs)"
8,0.307692,"(burgers, meatballs, eggs)"


## 協同過濾演算法(Collaborative Filtering) - 基於使用者

In [ ]:
from collections import defaultdict,Counter
import numpy as np
import math
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
store = [["burgers","meatballs","eggs"],
      ["turkey","burgers","mineral water","cooking oil"],
      ["burgers","grated cheese","pasta","honey","toothpaste"],
      ["meatballs","milk","eggs","honey","french fries","protein bar"],
      ["red wine","shrimp","pasta","pepper","eggs","chocolate","shampoo"],
      ["burgers","black tea","green tea"],
      ["burgers","black tea","green tea"],
      ["burgers","meatballs","eggs"],
      ["burgers","meatballs","eggs"],
      ["burgers","meatballs","eggs"],
      ["escalope","burgers","shallot"],
      ["red wine","shrimp","pasta","pepper","eggs","chocolate","shampoo"],
      ["spaghetti","milk","french fries","green tea"]]

In [ ]:
df = pd.DataFrame(apriori_Encoder_Array, columns=apriori_Encoder.columns_)
df

,black tea,burgers,chocolate,cooking oil,eggs,escalope,french fries,grated cheese,green tea,honey,meatballs,milk,mineral water,pasta,pepper,protein bar,red wine,shallot,shampoo,shrimp,spaghetti,toothpaste,turkey
0,False,True,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
1,False,True,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True
2,False,True,False,False,False,False,False,True,False,True,False,False,False,True,False,False,False,False,False,False,False,True,False
3,False,False,False,False,True,False,True,False,False,True,True,True,False,False,False,True,False,False,False,False,False,False,False
4,False,False,True,False,True,False,False,False,False,False,False,False,False,True,True,False,True,False,True,True,False,False,False
5,True,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
6,True,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
7,False,True,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
8,False,True,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
9,False,True,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False


In [ ]:
for u in df.columns:
  if df[u].dtype==bool:
    df[u]=df[u].astype('int')

In [ ]:
df

,black tea,burgers,chocolate,cooking oil,eggs,escalope,french fries,grated cheese,green tea,honey,meatballs,milk,mineral water,pasta,pepper,protein bar,red wine,shallot,shampoo,shrimp,spaghetti,toothpaste,turkey
0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
2,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0
3,0,0,0,0,1,0,1,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0
4,0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,1,0,0,0
5,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
8,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
9,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
a = df.iloc[0,].to_numpy().reshape(1, -1)
b = df.iloc[1,].to_numpy().reshape(1, -1)
cosine_similarity(a, b)

array([[0.28867513]])

In [ ]:
user_a = df.iloc[0,].to_numpy().reshape(1, -1)
new=[]
for i in range(0,12):
  b = df.iloc[i,].to_numpy().reshape(1, -1)
  similarity_matrix = cosine_similarity(user_a, b)
  new = np.append(new, similarity_matrix)

df.iloc[np.where(new)],new


(    black tea  burgers  chocolate  ...  spaghetti  toothpaste  turkey
 0           0        1          0  ...          0           0       0
 1           0        1          0  ...          0           0       1
 2           0        1          0  ...          0           1       0
 3           0        0          0  ...          0           0       0
 4           0        0          1  ...          0           0       0
 5           1        1          0  ...          0           0       0
 6           1        1          0  ...          0           0       0
 7           0        1          0  ...          0           0       0
 8           0        1          0  ...          0           0       0
 9           0        1          0  ...          0           0       0
 10          0        1          0  ...          0           0       0
 11          0        0          1  ...          0           0       0
 
 [12 rows x 23 columns],
 array([1.        , 0.28867513, 0.25819889, 0.4714

In [ ]:
#https://raw.githubusercontent.com/jerrywu2013/Hiskio_data/master/store_data.csv